In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.neural_network import MLPClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import confusion_matrix, accuracy_score

In [2]:
dataset = pd.read_csv('../CreatingDataset/TimeDomain/ElectricCurrentDataset.txt')

## Combinações de Correntes

In [3]:
classes=[]
for i in dataset['Device']:
    if i not in classes:
        classes.append(i)

#Transformando o Dataframe dataset em matriz
matrix_dataset = list(dataset.as_matrix())

#Transformando cada linha da matriz em um numpy array
for i in range(len(matrix_dataset)):
    matrix_dataset[i] = np.asarray(matrix_dataset[i])

#Dois a dois
for i in range(len(dataset)):
    for j in range(1, len(classes)):
        if i+(200*j)<len(dataset):
            matrix_dataset.append(matrix_dataset[i]+matrix_dataset[i+(200*j)])

#Três a três
num_classes = 4
for i in range(len(dataset)):
    for j in range(1, len(classes)):
        for k in range(1, num_classes):
            if i+(200*j)<len(dataset) and i+(200*(j+num_classes))<len(dataset):
                matrix_dataset.append(matrix_dataset[i] + matrix_dataset[i+(200*j)] + matrix_dataset[i+(200*(j+1))])

#Quatro a quatro
for i in range(200):
    matrix_dataset.append(matrix_dataset[i] + matrix_dataset[i+(200*1)] + matrix_dataset[i+(200*(2))] + matrix_dataset[i+(200*(3))])

/Users/jeanlima/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:7: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  import sys


### CurrentDataset - FrequencyDomain

In [8]:
device_position = 300
frequency_dataset = []
for i in range(len(matrix_dataset)):
    signal = []
    signal = list(matrix_dataset[i][:device_position])
    frequency_dataset.append(list(np.fft.fft(signal).real))
    frequency_dataset[i].append(matrix_dataset[i][device_position])
    

#Novo dataset
frequency_dataset = pd.DataFrame(frequency_dataset)
frequency_dataset.head()

,0,1,2,3,4,5,6,7,8,9,...,291,292,293,294,295,296,297,298,299,300
0,-3.93,-2.513044,-8.163932,33.629912,7.659697,6.725006,10.643506,13.177546,59.482317,-10.749250,...,-10.749250,59.482317,13.177546,10.643506,6.725006,7.659697,33.629912,-8.163932,-2.513044,Notebook
1,-1.00,-0.191735,-4.678745,23.926314,8.975643,6.839434,8.616491,10.477577,54.968947,-3.580283,...,-3.580283,54.968947,10.477577,8.616491,6.839434,8.975643,23.926314,-4.678745,-0.191735,Notebook
2,-3.07,-3.250578,-3.296642,28.901722,7.726107,9.185737,9.997531,13.401718,53.479286,-8.655162,...,-8.655162,53.479286,13.401718,9.997531,9.185737,7.726107,28.901722,-3.296642,-3.250578,Notebook
3,-1.94,-0.968233,-4.019366,27.971932,10.626979,7.588198,9.675171,14.644327,55.259634,-3.368294,...,-3.368294,55.259634,14.644327,9.675171,7.588198,10.626979,27.971932,-4.019366,-0.968233,Notebook
4,-4.44,-1.646007,-4.716094,32.419320,8.428120,8.383729,11.016293,12.865954,56.504946,-12.448456,...,-12.448456,56.504946,12.865954,11.016293,8.383729,8.428120,32.419320,-4.716094,-1.646007,Notebook


## Partição Holdout 80/20

In [9]:
y = frequency_dataset[device_position]
x = frequency_dataset.drop([device_position], axis=1)
X_train, X_test, Y_train, Y_test = train_test_split(x, y, test_size=0.20)

## Treinamento e Teste Centróides mais próximas

In [10]:
nc = NearestCentroid()
nc.fit(X_train, Y_train)
previsaoNC = nc.predict(X_test)

print('Acurácia:',accuracy_score(Y_test, previsaoNC))

Acurácia: 0.5159090909090909


## Treinamento e Teste KNN; K=5

In [11]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train, Y_train)

previsaokNN = knn.predict(X_test)

print('Acurácia:',accuracy_score(Y_test, previsaokNN))

Acurácia: 0.675


## Treino e Teste Gaussian Naive Bayes

In [16]:
gnb = GaussianNB()
gnb.fit(X_train, Y_train)

previsaoGNB = gnb.predict(X_test)

accuracy_score(Y_test, previsaoGNB)

0.8090909090909091

## Treinamento e Teste MLP

In [22]:
from math import ceil

#Regra da pirâmide Geométrica para estabelecer número máximo de neurônios
alfa = 0.5
input_neurons = 300
output_neurons = 4
num_neuronios = alfa*((input_neurons* output_neurons)**(1/2))
ceil(num_neuronios)

18

In [60]:
MLP = MLPClassifier(hidden_layer_sizes=(5,6,7),
                     max_iter=1000000, 
                     activation='identity', 
                     learning_rate='constant',
                     learning_rate_init=0.0005,
                     batch_size = 1,
                     solver='lbfgs')
MLP.fit(X_train, Y_train)
previsaoMLP = MLP.predict(X_test)

print('Acurácia:', accuracy_score(Y_test, previsaoMLP))

Acurácia: 0.8727272727272727
